In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from fastai.imports import *
from fastai.tabular import *
import pandas as pd
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier,ExtraTreesRegressor
from IPython.display import display
from sklearn import metrics
import os
import numpy as np

# Getting started

In [2]:
PATH = os.getcwd()
PATH

'/home/jakwisn/Documents/Machine Learning/Intro to machine learning'

In [10]:
! curl 'https://storage.googleapis.com/kaggle-competitions-data/kaggle/5407/205873/train.csv?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1569329094&Signature=j8BX6ctrw1hy%2BY%2BE6iTWcfuv2NteutP2vFRScT%2FMHBMJ9hoCO4XlUUlilPv3BqrX%2FxMHcLm6l68yu8ygCvSmZsCyZQr5%2FVYXXo4DEneZBcIjXKw4RS44socpy6jm8bhiDRBftt3BLmvxwv6gh2FvsivYNjS387dyH0qYMwmKltPeC%2Bpqnkm5yLcqCzwuogX%2Bw4ne9iLRB13PGXQcq0WDIWUrrKvHbP1wjDsJsYa77I3faXawvJ0JZwqWgb%2Bm9DQfB8lNz0KCspFwi6B8jpUzaSFqEvDeYM51AmietErPNJ87FY8ACgQY4Ql%2FJACsg%2BVrXghUQ6Jy5dJONJ6GALKJqw%3D%3D' -H 'User-Agent: Mozilla/5.0 (X11; Linux x86_64; rv:69.0) Gecko/20100101 Firefox/69.0' -H 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8' -H 'Accept-Language: en-US,en;q=0.5' --compressed -H 'Referer: https://www.kaggle.com/' -H 'Connection: keep-alive' -H 'Upgrade-Insecure-Requests: 1' -H 'TE: Trailers' -o housing_train.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   202  100   202    0     0   2082      0 --:--:-- --:--:-- --:--:--  2082


In [13]:
! ls

 cos.py			       housing_train.csv	 Untitled-1.py
 fastai07		      'notes - lesson 2.ipynb'	 Untitled2.ipynb
 Housing_Prices_Kaggle	       test.csv			 Untitled3.ipynb
 Housing_Prices_Kaggle_deep    train.csv		 Untitled.ipynb
 Housing_Prices_Kaggle_deep~   Untitled1.ipynb


In [14]:
PATH += '/housing_train.csv'
train = pd.read_csv('train.csv')

# display the data
train.head()
# we get lot's of collumns

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


# Preprocessing

In [15]:
# we evaluate as RMSE, so we take log first
train.Saleprice = np.log(train.SalePrice)
train.Saleprice[:5]

/home/jakwisn/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  


0    12.247694
1    12.109011
2    12.317167
3    11.849398
4    12.429216
Name: SalePrice, dtype: float64

In [16]:
# we have to change strings, objects to categorical variables 
train.MSZoning.unique(), train.Street.unique()

(array(['RL', 'RM', 'C (all)', 'FV', 'RH'], dtype=object),
 array(['Pave', 'Grvl'], dtype=object))

Therefore we will make function, that takes column and changes it's categorical content to integers

In [17]:
columns = []

for i in range(train.shape[1]): 
    if train.iloc[:,i].dtype == 'O': 
        columns.append(True)
    else : 
        columns.append(False)

cat_names = train.columns[columns]
columns = [not i for i in columns]
cont_names = train.columns[columns]


In [18]:
def categorize(X): 
    """
    X is column from Pandas Data Frame, therefore Series
    """
    # we get unique values from X
    values = X.unique() 

    # initially there are no nan's
    are_there_nans = False

    # we define shape of output
    out = np.zeros(X.size)
    
    for i in range(X.size):
        # in case of nan : 

        if pd.isnull(X[i]) : 
            out[i] = -1
            are_there_nans = True

        else: 
            k = X[i] == values
            out[i] = int(np.where(k == True)[0][0])
    
    if are_there_nans : 
        out += 1   
    
    return out
    
def mean_nans(X): 
    X = np.array(X)
    non_nan = []
    for i in range(X.size): 
        if not pd.isnull(X[i]) : 
            non_nan.append(X[i])
    
    non_nan = np.array(non_nan)
    nan_positions = np.where(pd.isnull(X) == True)[0]
    X[nan_positions] = non_nan.mean()
    return pd.Series(X)
    
def categorize_and_mean_nans(DataFrame): 
    
    m,n = DataFrame.shape
    
    for i in range(n): 
        if DataFrame.iloc[:,i].dtype == 'O' : 
            DataFrame.iloc[:,i] = categorize(DataFrame.iloc[:,i])
        else : 
            DataFrame.iloc[:,i] = mean_nans(DataFrame.iloc[:,i])
    return DataFrame
    

In [19]:
train_ = categorize_and_mean_nans(train)
train_.head(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,0.0,65.000000,8450,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,2,2008,0.0,0.0,208500
1,2,20,0.0,80.000000,9600,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,5,2007,0.0,0.0,181500
2,3,60,0.0,68.000000,11250,0.0,0.0,1.0,0.0,0.0,...,0,0.0,0.0,0.0,0,9,2008,0.0,0.0,223500
3,4,70,0.0,60.000000,9550,0.0,0.0,1.0,0.0,0.0,...,0,0.0,0.0,0.0,0,2,2006,0.0,1.0,140000
4,5,60,0.0,84.000000,14260,0.0,0.0,1.0,0.0,0.0,...,0,0.0,0.0,0.0,0,12,2008,0.0,0.0,250000
5,6,50,0.0,85.000000,14115,0.0,0.0,1.0,0.0,0.0,...,0,0.0,2.0,2.0,700,10,2009,0.0,0.0,143000
6,7,20,0.0,75.000000,10084,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,8,2007,0.0,0.0,307000
7,8,60,0.0,70.049958,10382,0.0,0.0,1.0,0.0,0.0,...,0,0.0,0.0,2.0,350,11,2009,0.0,0.0,200000
8,9,50,1.0,51.000000,6120,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,4,2008,0.0,1.0,129900
9,10,190,0.0,50.000000,7420,0.0,0.0,0.0,0.0,0.0,...,0,0.0,0.0,0.0,0,1,2008,0.0,0.0,118000


In [24]:
# n_jobs splits work between cores
m = RandomForestRegressor(n_estimators=10000,
                        min_samples_leaf=3,
                        max_features= 0.3,
                        n_jobs = -1,
                        verbose = 1)
m.fit(train_.drop('SalePrice', axis = 1), train_.SalePrice) 
m.score(train_.drop('SalePrice', axis = 1), train.SalePrice)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:    0.4s
[Parallel(n_jobs=-1)]: Done 442 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 792 tasks      | elapsed:    1.4s
[Parallel(n_jobs=-1)]: Done 1242 tasks      | elapsed:    2.3s
[Parallel(n_jobs=-1)]: Done 1792 tasks      | elapsed:    3.3s
[Parallel(n_jobs=-1)]: Done 2442 tasks      | elapsed:    4.5s
[Parallel(n_jobs=-1)]: Done 3192 tasks      | elapsed:    5.9s
[Parallel(n_jobs=-1)]: Done 4042 tasks      | elapsed:    7.4s
[Parallel(n_jobs=-1)]: Done 4992 tasks      | elapsed:    9.1s
[Parallel(n_jobs=-1)]: Done 6042 tasks      | elapsed:   11.0s
[Parallel(n_jobs=-1)]: Done 7192 tasks      | elapsed:   13.2s
[Parallel(n_jobs=-1)]: Done 8442 tasks      | elapsed:   15.5s
[Parallel(n_jobs=-1)]: Done 9792 tasks      | elapsed:   18.0s
[Parallel(n_jobs=-1)]: Done 10000 out of 

0.9567738935474193

In [25]:
test = pd.read_csv('test.csv')
test = categorize_and_mean_nans(test)
Result = m.predict(test)


[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  42 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 192 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 442 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 792 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 1242 tasks      | elapsed:    0.1s
[Parallel(n_jobs=4)]: Done 1792 tasks      | elapsed:    0.2s
[Parallel(n_jobs=4)]: Done 2442 tasks      | elapsed:    0.4s
[Parallel(n_jobs=4)]: Done 3192 tasks      | elapsed:    0.5s
[Parallel(n_jobs=4)]: Done 4042 tasks      | elapsed:    0.6s
[Parallel(n_jobs=4)]: Done 4992 tasks      | elapsed:    0.7s
[Parallel(n_jobs=4)]: Done 6042 tasks      | elapsed:    0.8s
[Parallel(n_jobs=4)]: Done 7192 tasks      | elapsed:    0.9s
[Parallel(n_jobs=4)]: Done 8442 tasks      | elapsed:    1.1s
[Parallel(n_jobs=4)]: Done 9792 tasks      | elapsed:    1.3s
[Parallel(n_jobs=4)]: Done 10000 out of 10000 | elapsed:

In [26]:
Id = np.array(test.Id)
Result_ = pd.DataFrame({'Id' : Id, 'SalePrice' :Result})

Result_.to_csv('Housing_Prices_Kaggle_tree', index = False)